In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_file, show, output_notebook
from bokeh.models import CustomJS
from bokeh.models.widgets import CheckboxGroup
from bokeh.layouts import row
from bokeh.palettes import Viridis4
from bokeh.models.annotations import Title, Legend


In [2]:
#Read in the original data series for the 3 crimes
misdemeanor = pd.read_csv('df_misdemeanor.csv', index_col=0)
felony = pd.read_csv('df_felony.csv', index_col=0)
violation = pd.read_csv('df_violation.csv', index_col=0)

In [4]:
misdemeanor.head()

,quantity
date,
2006-01-01,235
2006-01-08,1170
2006-01-15,1167
2006-01-22,1197
2006-01-29,1241


In [11]:
misdemeanor.reset_index(inplace=True)
felony.reset_index(inplace=True)
violation.reset_index(inplace=True)

In [5]:
#Read in the forecasts of the 3 crimes
misdemeanor_forecast = pd.read_csv('misdemeanor_forecast.csv', index_col=0)
felony_forecast = pd.read_csv('felony_forecast.csv', index_col=0)
violation_forecast = pd.read_csv('violation_forecast.csv', index_col=0)

In [7]:
misdemeanor_forecast.index.name = 'date'
felony_forecast.index.name = 'date'
violation_forecast.index.name = 'date'

In [12]:
misdemeanor_forecast.reset_index(inplace=True)
felony_forecast.reset_index(inplace=True)
violation_forecast.reset_index(inplace=True)

In [13]:
misdemeanor_forecast.head()

,date,number_of_crimes
0,2018-01-07,1031.390556
1,2018-01-14,1204.736685
2,2018-01-21,1147.067653
3,2018-01-28,1189.918999
4,2018-02-04,1183.821501


In [23]:
felony_forecast.head()

,date,quantity
0,2018-01-07,525.734767
1,2018-01-14,575.948898
2,2018-01-21,562.696192
3,2018-01-28,618.650355
4,2018-02-04,573.487693


In [15]:
#Setting the date for all the data to be in the same format
misdemeanor['date'] = pd.to_datetime(misdemeanor['date'], format='%Y/%m/%d')
felony['date'] = pd.to_datetime(felony['date'], format='%Y/%m/%d')
violation['date'] = pd.to_datetime(violation['date'], format='%Y/%m/%d')
misdemeanor_forecast['date'] = pd.to_datetime(misdemeanor_forecast['date'], format='%Y/%m/%d')
felony_forecast['date'] = pd.to_datetime(felony_forecast['date'], format='%Y/%m/%d')
violation_forecast['date'] = pd.to_datetime(violation_forecast['date'], format='%Y/%m/%d')

In [60]:
#Using Bokeh to plot the data series for the 3 crimes and their forecasts
#Each crime and its respective forecast has its own plot and checkbox to toggle

p = figure(x_axis_type='datetime', plot_width=1200, plot_height=600, title='Manhattan Crime')
aline = p.line(misdemeanor['date'], misdemeanor['quantity'], line_width=2, color='red')
bline = p.line(misdemeanor_forecast['date'], misdemeanor_forecast['number_of_crimes'], line_width=2, color=Viridis4[1])
cline = p.line(felony['date'], felony['quantity'], line_width=2, color='blue')
dline = p.line(felony_forecast['date'], felony_forecast['quantity'], line_width=2, color=Viridis4[1])
eline = p.line(violation['date'], violation['quantity'], line_width=2, color='green')
fline = p.line(violation_forecast['date'], violation_forecast['quantity'], line_width=2, color=Viridis4[1])

p.yaxis.axis_label = 'Number of Crimes'
p.xaxis.axis_label = 'Time Span'

legend = Legend(items=[
    ("Misdemeanors",   [aline]),
    ("Misdemeanors Forecast", [bline]),
    ("Felonies", [cline]),
    ("Felonies Forecast", [dline]),
    ("Violations", [eline]),
    ("Violations Forecast", [fline])]
                ,location=(0, 200))

# t = Title()
# t.text = 'Manhattan Crime'
# p.title = t
p.title.text_color = Viridis4[1]
p.title.text_font = 'helvetica'
p.title.text_font_style = 'bold'
p.title.text_font_size = '20pt'
p.title.align = 'center'


p.add_layout(legend, 'right')

checkboxes = CheckboxGroup(labels=list(['Misdemeanors', 'Misdemeanors Forecast', 'Felonies', 
                                        'Felonies Forecast', 'Violations', 'Violations Forecast']), 
                           active=[0, 1, 2, 3, 4, 5])
callback = CustomJS(code="""aline.visible = false;
                            bline.visible = false;
                            cline.visible = false;
                            dline.visible = false;
                            eline.visible = false;
                            fline.visible = false;
                            
                            // cb_obj is injected in thanks to the callback
                            if (cb_obj.active.includes(0)){aline.visible = true;} 
                                // 0 index box is aline
                            if (cb_obj.active.includes(1)){bline.visible = true;}
                            if (cb_obj.active.includes(2)){cline.visible = true;}
                            if (cb_obj.active.includes(3)){dline.visible = true;}
                            if (cb_obj.active.includes(4)){eline.visible = true;}
                            if (cb_obj.active.includes(5)){fline.visible = true;}
                            """,
                    args={'aline': aline, 'bline': bline, 'cline': cline, 'dline': dline, 'eline': eline, 'fline': fline,})
checkboxes.js_on_click(callback)
output_file('ManhattanCrime.html')
show(row(p, checkboxes))